# 🧠 **Evaluación de Modelos Supervisados**
Este notebook trata sobre cómo evaluar modelos de aprendizaje supervisado. Es decir, modelos que aprenden a predecir una variable de interés (por ejemplo, precio, ingresos, probabilidad de impago, etc.) a partir de otras variables (predictoras o independientes).
El objetivo principal es: Comparar el rendimiento de distintos modelos supervisados (regresión o clasificación) utilizando métricas de evaluación apropiadas.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score, roc_curve


In [3]:
datos = pd.read_csv("credit_customers.csv")

# 📊 **Conozcamos nuestra base de datos** 
Primero realizaremos un mini EDA ya que es super importante conocer nuestros datos, saber como son las variables he identificar lo que debemos mejorar

In [4]:
datos

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,...,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker,class
0,<0,6.0,critical/other existing credit,radio/tv,1169.0,no known savings,>=7,4.0,male single,none,...,real estate,67.0,none,own,2.0,skilled,1.0,yes,yes,good
1,0<=X<200,48.0,existing paid,radio/tv,5951.0,<100,1<=X<4,2.0,female div/dep/mar,none,...,real estate,22.0,none,own,1.0,skilled,1.0,none,yes,bad
2,no checking,12.0,critical/other existing credit,education,2096.0,<100,4<=X<7,2.0,male single,none,...,real estate,49.0,none,own,1.0,unskilled resident,2.0,none,yes,good
3,<0,42.0,existing paid,furniture/equipment,7882.0,<100,4<=X<7,2.0,male single,guarantor,...,life insurance,45.0,none,for free,1.0,skilled,2.0,none,yes,good
4,<0,24.0,delayed previously,new car,4870.0,<100,1<=X<4,3.0,male single,none,...,no known property,53.0,none,for free,2.0,skilled,2.0,none,yes,bad
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,no checking,12.0,existing paid,furniture/equipment,1736.0,<100,4<=X<7,3.0,female div/dep/mar,none,...,real estate,31.0,none,own,1.0,unskilled resident,1.0,none,yes,good
996,<0,30.0,existing paid,used car,3857.0,<100,1<=X<4,4.0,male div/sep,none,...,life insurance,40.0,none,own,1.0,high qualif/self emp/mgmt,1.0,yes,yes,good
997,no checking,12.0,existing paid,radio/tv,804.0,<100,>=7,4.0,male single,none,...,car,38.0,none,own,1.0,skilled,1.0,none,yes,good
998,<0,45.0,existing paid,radio/tv,1845.0,<100,1<=X<4,4.0,male single,none,...,no known property,23.0,none,for free,1.0,skilled,1.0,yes,yes,bad


In [5]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   checking_status         1000 non-null   object 
 1   duration                1000 non-null   float64
 2   credit_history          1000 non-null   object 
 3   purpose                 1000 non-null   object 
 4   credit_amount           1000 non-null   float64
 5   savings_status          1000 non-null   object 
 6   employment              1000 non-null   object 
 7   installment_commitment  1000 non-null   float64
 8   personal_status         1000 non-null   object 
 9   other_parties           1000 non-null   object 
 10  residence_since         1000 non-null   float64
 11  property_magnitude      1000 non-null   object 
 12  age                     1000 non-null   float64
 13  other_payment_plans     1000 non-null   object 
 14  housing                 1000 non-null   o

In [6]:
datos.describe()

,duration,credit_amount,installment_commitment,residence_since,age,existing_credits,num_dependents
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,20.903000,3271.258000,2.973000,2.845000,35.546000,1.407000,1.155000
std,12.058814,2822.736876,1.118715,1.103718,11.375469,0.577654,0.362086
min,4.000000,250.000000,1.000000,1.000000,19.000000,1.000000,1.000000
25%,12.000000,1365.500000,2.000000,2.000000,27.000000,1.000000,1.000000
50%,18.000000,2319.500000,3.000000,3.000000,33.000000,1.000000,1.000000
75%,24.000000,3972.250000,4.000000,4.000000,42.000000,2.000000,1.000000
max,72.000000,18424.000000,4.000000,4.000000,75.000000,4.000000,2.000000


In [9]:
datos.describe(include="object")

,checking_status,credit_history,purpose,savings_status,employment,personal_status,other_parties,property_magnitude,other_payment_plans,housing,job,own_telephone,foreign_worker,class
count,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000
unique,4,5,10,5,5,4,3,4,3,3,4,2,2,2
top,no checking,existing paid,radio/tv,<100,1<=X<4,male single,none,car,none,own,skilled,none,yes,good
freq,394,530,280,603,339,548,907,332,814,713,630,596,963,700


 ### 🧾 **¿Qué variables tiene nuestra base de datos?**
- `checking_status`: Estado de la cuenta corriente del cliente. Indica la solvencia inicial.
- `duration`: Duración del crédito en meses.
- `credit_history`: Historial crediticio previo (si tiene pagos a tiempo, retrasos, etc.).
- `purpose`: Propósito del crédito.
- `credit_amount`: Monto total del crédito solicitado.
- `savings_status`: Estado de cuenta de ahorro del cliente. Similar a checking_status pero para ahorros.
- `employment`: Años en el empleo actual.
- `installment_commitment`: Porcentaje del ingreso mensual comprometido al pago del crédito.
- `personal_status`: Estado civil y género.
- `other_parties`: Si hay otros garantes en el crédito.
- `residence_since`: Años viviendo en la residencia actual.
- `property_magnitude`: Tipo de propiedad que posee.
- `age`: Edad del solicitante.
- `other_payment_plans`: Si tiene otros planes de pago.
- `housing`: Tipo de vivienda.
- `existing_credits`: Número de créditos existentes en otras instituciones.
- `job`: Tipo de trabajo.
- `num_dependents`: Número de personas a cargo.
- `own_telephone`: Si tiene teléfono.
- `foreign_worker`: Si es trabajador extranjero.
- `class`: Indica si se considera un cliente de buen o mal riesgo crediticio.

#### 🎯 *¿Qué es la variable objetivo?*
 La variable objetivo (también llamada `target` o `etiqueta`) es lo que queremos predecir. Es la respuesta que el modelo aprende a estimar a partir de otras variables.

#### 🧠 **¿Por qué class es la variable objetivo?**
Un banco quiere predecir si un cliente es un buen o mal riesgo antes de otorgarle un préstamo. La columna class tiene dos valores:
- "good": cliente con buen comportamiento crediticio (probablemente paga sus deudas a tiempo).
- "bad": cliente con mal comportamiento crediticio (riesgo de impago).
 Es una etiqueta categórica: Como solo tiene dos clases (good y bad), se trata de un problema de clasificación binaria, muy típico en modelos supervisados.
 Además, todas las demás variables describen al cliente, las otras columnas (edad, ingreso, historial crediticio, etc.) son características que el modelo usará para predecir esa etiqueta. Ninguna otra columna representa un resultado o decisión final, como sí lo hace class.

##### 💡 El tipo de pregunta que se hace un economista o analista de crédito: "Dado un conjunto de características del cliente... ¿es riesgoso prestarle dinero?"

# 🔍 **Preprocesamiento de Datos**
Debemos convertir los datos en un formato limpio y numérico que los modelos puedan entender y procesar.
1. Separar la variable objetivo del resto de los datos:
- X será el conjunto de variables independientes (predictoras).
- y será la variable dependiente (class).
2. Luego separaremos los datos  donde el 20% de los datos se usan para prueba y el 80% restante para entrenamiento. Esto con el fin de entrenar el modelo con unos datos (conocidos) y evaluarlo con otros que no ha visto.

In [7]:
X = datos.drop(columns="class")
y = datos["class"]


In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.25,
    random_state=42
)

# 🏗️ **Pipelines**
Dado que tenemos muchas variables numéricas, haremos transformaciones que nos faciliten la transformación de los datos. Los pipelines de preprocesamiento en `scikit-learn` son súper útiles porque te permiten automatizar todo el proceso de limpieza y transformación de datos, asegurándote de que se aplique exactamente lo mismo tanto a los datos de entrenamiento como a los de prueba. Además, las variables numéricas también se les hará una transformación donde se estandarizará la variable, convirtiendolas a media 0 y desviación estándar 1, esto es útil, por ejemplo, para modelos como regresión logística, que se ven muy afectados por la escala de los datos.

In [9]:
cat_cols = X_train.select_dtypes(include="object").columns.tolist()

num_cols = X_train.select_dtypes(exclude="object").columns.tolist()

In [10]:
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(drop='first', handle_unknown='ignore'))
])


In [11]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_cols),
        ('cat', categorical_transformer, cat_cols)
    ]
)

In [12]:
X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

# **Empecemos con los modelos**
Ahora que ya están los datos preprocesados correctamente, podemos relaizar los modelos supervisados. 
Recordemos que tenemos un problema declasificación binaria, donde queremos predecir si un cliente es "good" o "bad" en función de sus características.

### Trabajaremos 3 modelos: 
1️⃣ **Regresión logística:** Predice la probabilidad de que una observación pertenezca a una clase, por ejemplo, que un cliente sea "bueno". Usa una función sigmoide para asegurar que las predicciones estén entre 0 y 1.
##### ✅ Ventajas:
- Muy rápida y fácil de interpretar.
- Ideal como modelo base.
- Funciona bien si los datos son linealmente separables.
##### ⚠️ Desventajas:
- No capta relaciones no lineales.
- Supone independencia entre variables.

2️⃣ **Árbol de decisión:** Este modelo clasifica dividiendo el espacio de datos con preguntas. Además genera una estructura de árbol con nodos de decisión y hojas que representan una clase.
##### ✅ Ventajas:
- Muy interpretables: fácil explicar las decisiones.
- Capturan relaciones no lineales y de interacción.
- No necesitan escalado
##### ⚠️ Desventajas:
- Sensibles a pequeñas variaciones en los datos.

3️⃣ **K-NEAREST NEIGHBORS (KNN):** Clasifica un dato nuevo mirando sus k vecinos más cercanos y eligiendo la clase más común entre ellos.
##### ✅ Ventajas:
- No necesita entrenamiento como tal (lazy learner).
- Funciona bien con fronteras de decisión complejas.
##### ⚠️ Desventajas:
- Muy sensible al escalado de variables.
- Costoso computacionalmente con grandes volúmenes de datos.
- Requiere ajustar el hiperparámetro k.

# 🔵 Regresión Logística

In [13]:
# Entrenar modelo
log_model = LogisticRegression()
log_model.fit(X_train_transformed, y_train)

# Predecir
y_pred_log = log_model.predict(X_test_transformed)

# Evaluar
print("Regresión Logística:")
print(confusion_matrix(y_test, y_pred_log))
print(classification_report(y_test, y_pred_log))

Regresión Logística:
[[ 36  36]
 [ 21 157]]
              precision    recall  f1-score   support

         bad       0.63      0.50      0.56        72
        good       0.81      0.88      0.85       178

    accuracy                           0.77       250
   macro avg       0.72      0.69      0.70       250
weighted avg       0.76      0.77      0.76       250



# 🌳 Árbol de Decisión

In [14]:
tree_model = DecisionTreeClassifier(random_state=42)
tree_model.fit(X_train_transformed, y_train)

y_pred_tree = tree_model.predict(X_test_transformed)

print("Árbol de Decisión:")
print(confusion_matrix(y_test, y_pred_tree))
print(classification_report(y_test, y_pred_tree))


Árbol de Decisión:
[[ 34  38]
 [ 33 145]]
              precision    recall  f1-score   support

         bad       0.51      0.47      0.49        72
        good       0.79      0.81      0.80       178

    accuracy                           0.72       250
   macro avg       0.65      0.64      0.65       250
weighted avg       0.71      0.72      0.71       250



# 👟 K-Nearest Neighbors

In [15]:
knn_model = KNeighborsClassifier(n_neighbors=5)  # puedes probar con otros valores de k
knn_model.fit(X_train_transformed, y_train)

y_pred_knn = knn_model.predict(X_test_transformed)

print("K-Nearest Neighbors:")
print(confusion_matrix(y_test, y_pred_knn))
print(classification_report(y_test, y_pred_knn))

K-Nearest Neighbors:
[[ 27  45]
 [ 21 157]]
              precision    recall  f1-score   support

         bad       0.56      0.38      0.45        72
        good       0.78      0.88      0.83       178

    accuracy                           0.74       250
   macro avg       0.67      0.63      0.64       250
weighted avg       0.72      0.74      0.72       250



**Definamos los modelos**

In [16]:
modelo_knn = KNeighborsClassifier()
modelo_lineal = LogisticRegression(max_iter=1000)
decision_tree = DecisionTreeClassifier(max_depth=4)

**Entrenemos los modelos**

In [17]:
modelo_knn.fit(X_train_transformed, y_train)
modelo_lineal.fit(X_train_transformed, y_train)
decision_tree.fit(X_train_transformed, y_train)

DecisionTreeClassifier(max_depth=4)

**Evaluemos los modelos**

In [18]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd

# Función de evaluación para modelos de clasificación
def evaluate_model_classification(model, X, y):
    y_pred = model.predict(X)
    return {
        "accuracy": accuracy_score(y, y_pred),
        "precision": precision_score(y, y_pred, average="weighted"),
        "recall": recall_score(y, y_pred, average="weighted"),
        "f1_score": f1_score(y, y_pred, average="weighted"),
    }

# Evaluar cada modelo en el conjunto de entrenamiento
knn_eval = evaluate_model_classification(modelo_knn, X_train_transformed, y_train)
log_eval = evaluate_model_classification(modelo_lineal, X_train_transformed, y_train)
dt_eval = evaluate_model_classification(decision_tree, X_train_transformed, y_train)

# Convertir resultados a DataFrame
df_eval = pd.DataFrame([knn_eval, log_eval, dt_eval], index=["KNN", "Logistic", "DecisionTree"])
df_eval.reset_index(inplace=True)
df_eval.rename(columns={"index": "model"}, inplace=True)

df_eval


,model,accuracy,precision,recall,f1_score
0,KNN,0.804000,0.799400,0.804000,0.790821
1,Logistic,0.780000,0.771439,0.780000,0.772502
2,DecisionTree,0.750667,0.740023,0.750667,0.742618


### 📊 **Interpretación de los resultados**
##### **Accuracy**: Proporción total de predicciones correctas.
  - KNN tiene la mayor precisión general del 80.4%.
##### **Precision:** Qué tan precisas fueron las predicciones positivas, teniendo en cuenta el desequilibrio de clases.
  - KNN nuevamente lidera con 79.9%, lo que indica que comete menos falsos positivos.
##### **Recall:** Cuántos casos positivos reales fueron correctamente identificados.
  - El valor de recall de KNN y logistic es bastante cercano, lo cual es bueno si te importa no dejar pasar casos verdaderos.
##### **F1 Score:** Media armónica entre precision y recall. Es especialmente útil cuando tienes un desbalance de clases.
  - KNN tiene el mejor equilibrio entre precisión y sensibilidad.
## 🏁 **Conclusión:**
- KNN es el mejor modelo en tu conjunto de entrenamiento, según todas las métricas.
- Logistic Regression no está muy lejos, y puede ser preferible si buscas un modelo más interpretable.
- Decision Tree tiene el rendimiento más bajo en todos los frentes, lo cual puede ser una señal de sobreajuste o de que requiere más ajuste de hiperparámetros.



In [22]:
from sklearn.model_selection import cross_val_score
import numpy as np

# Validación cruzada con k=5, usando accuracy como métrica
cv_scores_knn = cross_val_score(modelo_knn, X_train_transformed, y_train, cv=5, scoring="accuracy")
cv_scores_linear = cross_val_score(modelo_lineal, X_train_transformed, y_train, cv=5, scoring="accuracy")
cv_scores_dt = cross_val_score(decision_tree, X_train_transformed, y_train, cv=5, scoring="accuracy")

# Mostrar resultados
print("\nCross-Validation Accuracy Scores:")
print("KNN:", cv_scores_knn)
print("Logistic Regression:", cv_scores_linear)
print("Decision Tree:", cv_scores_dt)

# Promedio de accuracy
print("\nAverage Accuracy Scores:")
print("KNN:", np.mean(cv_scores_knn))
print("Logistic Regression:", np.mean(cv_scores_linear))
print("Decision Tree:", np.mean(cv_scores_dt))


Cross-Validation Accuracy Scores:
KNN: [0.78       0.68       0.68666667 0.67333333 0.73333333]
Logistic Regression: [0.76666667 0.73333333 0.64666667 0.80666667 0.71333333]
Decision Tree: [0.68666667 0.66       0.66       0.70666667 0.62666667]

Average Accuracy Scores:
KNN: 0.7106666666666667
Logistic Regression: 0.7333333333333333
Decision Tree: 0.6679999999999999


### 📊 **Interpretación de los resultados**
- El accuracy varió entre 67.3% y 78% en distintas particiones del dataset.
## 🏁 **Conclusión:**
La regresión logística tiene el mejor desempeño promedio en accuracy, por lo que podría considerarse el modelo más robusto de los tres, al menos según esta métrica.

In [23]:
# Agregar resultados de validación cruzada (accuracy promedio y desviación estándar)
df_eval["cv_accuracy_avg"] = [cv_scores_knn.mean(), cv_scores_linear.mean(), cv_scores_dt.mean()]
df_eval["cv_accuracy_std"] = [cv_scores_knn.std(), cv_scores_linear.std(), cv_scores_dt.std()]

df_eval

,model,accuracy,precision,recall,f1_score,cv_accuracy_avg,cv_accuracy_std
0,KNN,0.804000,0.799400,0.804000,0.790821,0.710667,0.040574
1,Logistic,0.780000,0.771439,0.780000,0.772502,0.733333,0.053666
2,DecisionTree,0.750667,0.740023,0.750667,0.742618,0.668000,0.027129


### 📊 **Interpretación de los resultados**
- KNN domina en las métricas de entrenamiento (accuracy, precision, recall y F1-score), lo cual sugiere buen ajuste a los datos de entrenamiento.
- Logistic Regression gana en la métrica de validación cruzada (CV), lo que indica mejor capacidad de generalización.
- Decision Tree tiene la menor desviación estándar en CV, lo cual implica un desempeño más estable… pero con menor accuracy general.

In [33]:
# Evaluar los modelos en el conjunto de prueba
knn_test_eval = evaluate_model_classification(modelo_knn, X_test_transformed, y_test)
log_test_eval = evaluate_model_classification(modelo_lineal, X_test_transformed, y_test)
dt_test_eval = evaluate_model_classification(decision_tree, X_test_transformed, y_test)

# Agregar resultados de test al DataFrame
df_eval["test_accuracy"] = [knn_test_eval["accuracy"], log_test_eval["accuracy"], dt_test_eval["accuracy"]]
df_eval["test_f1"] = [knn_test_eval["f1_score"], log_test_eval["f1_score"], dt_test_eval["f1_score"]]

df_eval


,model,accuracy,precision,recall,f1_score,cv_accuracy_avg,cv_accuracy_std,test_accuracy,test_f1
0,KNN,0.804000,0.799400,0.804000,0.790821,0.710667,0.040574,0.736,0.717937
1,Logistic,0.780000,0.771439,0.780000,0.772502,0.733333,0.053666,0.772,0.763353
2,DecisionTree,0.750667,0.740023,0.750667,0.742618,0.666667,0.026331,0.720,0.718810


### 📊 **Interpretación de los resultados**
- KNN tiene el mejor desempeño en entrenamiento, pero pierde fuerza en test (0.736) y validación cruzada, lo que puede sugerir overfitting, es decir, el modelo aprende demasiado bien los datos de entrenamiento, incluyendo el ruido, errores o patrones específicos que no se repiten en nuevos datos
- Logistic Regression tiene un excelente balance: no es el mejor en training, pero gana en validación cruzada y test, lo cual la hace la opción más robusta para generalizar a nuevos datos.
- Decision Tree es el más estable (menor std), pero con el peor desempeño general.
## 🏁 **Conclusión:**
El modelo más recomendable es la regresión logística, ya que logra el mejor desempeño en el conjunto de prueba, mejor F1, y mejor desempeño promedio en validación cruzada, lo que indica que generaliza mejor a datos nuevos.